In [2]:
import pandas as pd
import glob

# 병합할 파일 목록 (파일 이름 패턴을 사용하거나, 리스트로 직접 지정)
# 4개의 파일 이름이 query (숫자).csv 형태이므로 glob을 사용하면 편리합니다.
file_list = ['query (7).csv', 'query (6).csv', 'query (5).csv', 'query (4).csv']

# 또는 glob.glob("query (*).csv") 를 사용할 수도 있습니다.

# 유지할 컬럼 목록
columns_to_keep = ['time', 'latitude', 'longitude', 'depth', 'nst', 'gap', 'mag']

# 각 파일을 읽어와서 리스트에 저장
all_dataframes = []
for filename in file_list:
    try:
        df = pd.read_csv(filename)

        # 파일에 필요한 컬럼이 모두 있는지 확인
        if all(col in df.columns for col in columns_to_keep):
            # 필요한 컬럼만 선택
            df_selected = df[columns_to_keep]
            all_dataframes.append(df_selected)
            print(f"'{filename}' 처리 완료 (행: {len(df_selected)})")
        else:
            print(f"경고: '{filename}'에 필요한 컬럼이 부족하여 건너뜁니다.")

    except Exception as e:
        print(f"'{filename}' 처리 중 오류 발생: {e}")

# 모든 데이터프레임을 하나로 합치기 (위아래로 붙이기)
if all_dataframes:
    combined_df = pd.concat(all_dataframes, ignore_index=True)

    # 결과 파일로 저장
    output_filename = 'combined_earthquake_data.csv'
    combined_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"\n총 {len(combined_df)}개의 행이 '{output_filename}' 파일로 저장되었습니다.")
else:
    print("\n병합할 데이터가 없습니다. 파일 이름이나 컬럼명을 확인해주세요.")

'query (7).csv' 처리 완료 (행: 6768)
'query (6).csv' 처리 완료 (행: 6323)
'query (5).csv' 처리 완료 (행: 5393)
'query (4).csv' 처리 완료 (행: 4023)

총 22507개의 행이 'combined_earthquake_data.csv' 파일로 저장되었습니다.


In [3]:
import pandas as pd

# 입력 파일 (TSV)
input_file = 'tsunamis-2025-11-03_13-35-46_+0900.tsv'

# 출력 파일 (CSV)
output_file = 'renamed_tsunami_data.csv'

# 변경할 컬럼 이름 맵핑
# { '기존 컬럼명' : '새 컬럼명' }
rename_map = {
    'Earthquake Magnitude': 'mag',
    'Tsunami Cause Code': 'tsunami',
    'Latitude': 'latitude',
    'Longitude': 'longitude'
}

try:
    # 1. TSV 파일을 읽어옵니다. (구분자=탭)
    #    (파일 구조 확인 후 header=0, skiprows=[1] 사용)
    df = pd.read_csv(input_file, sep='\t', header=0, skiprows=[1])

    # 컬럼 이름 앞뒤에 붙어있을 수 있는 따옴표 제거
    df.columns = df.columns.str.strip('"')

    # 2. 지정된 컬럼명 변경
    df.rename(columns=rename_map, inplace=True)

    # 3. 변경된 DataFrame을 새 CSV 파일로 저장 (기본값=쉼표 구분)
    df.to_csv(output_file, index=False, encoding='utf-8-sig')

    print(f"파일 처리 완료! '{output_file}' (CSV 파일)로 저장되었습니다.")

except FileNotFoundError:
    print(f"오류: '{input_file}'을(를) 찾을 수 없습니다. 파일이 코드와 같은 위치에 있는지 확인하세요.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

파일 처리 완료! 'renamed_tsunami_data.csv' (CSV 파일)로 저장되었습니다.


In [4]:
import pandas as pd

# 입력 파일
input_file = 'renamed_tsunami_data.csv'
# 저장할 파일
output_file = 'filtered_tsunami_data.csv'

# 1. 유지할 컬럼 목록
columns_to_keep = [
    'Year',
    'Mo',
    'tsunami',
    'mag',
    'latitude',
    'longitude',
    'Tsunami Event Validity'
]

# 2. 필터링할 컬럼과 값
filter_column = 'Tsunami Event Validity'
valid_values = [3, 4] # 3 또는 4 인 값만 유지

try:
    # 파일 읽기
    df = pd.read_csv(input_file)

    # 3. 지정된 컬럼만 선택 (유지)
    # 리스트에 없는 모든 컬럼은 삭제됩니다.
    df_selected = df[columns_to_keep]

    # 4. 'Tsunami Event Validity' 컬럼 값이 3 또는 4인 행만 필터링
    # .isin() 함수를 사용하여 [3, 4] 리스트에 포함된 값만 확인
    df_filtered = df_selected[df_selected[filter_column].isin(valid_values)]

    # 5. 새 CSV 파일로 저장
    df_filtered.to_csv(output_file, index=False, encoding='utf-8-sig')

    print(f"처리 완료! '{output_file}'로 저장되었습니다.")
    print(f"원본 행: {len(df)}, 컬럼 선택 후: {len(df_selected)}, 최종 필터링 후: {len(df_filtered)}")
    print("\n필터링된 데이터 (상위 5개):")
    print(df_filtered.head())

except FileNotFoundError:
    print(f"오류: '{input_file}'을(를) 찾을 수 없습니다.")
except KeyError as e:
    # columns_to_keep 리스트에 없는 컬럼명을 요청했을 때 발생
    print(f"오류: {e} 컬럼을 찾을 수 없습니다. 컬럼명을 확인하세요.")
except Exception as e:
    print(f"알 수 없는 오류가 발생했습니다: {e}")

처리 완료! 'filtered_tsunami_data.csv'로 저장되었습니다.
원본 행: 135, 컬럼 선택 후: 135, 최종 필터링 후: 130

필터링된 데이터 (상위 5개):
   Year  Mo  tsunami  mag  latitude  longitude  Tsunami Event Validity
0  2001   1        1  7.7    13.049    -88.660                       3
1  2001   6        1  8.4   -16.265    -73.641                       4
2  2001  10        1  6.1    52.630   -132.200                       4
3  2001  12        1  6.8    23.954    122.734                       4
4  2002   1        1  7.2   -17.600    167.856                       4


In [5]:
import pandas as pd

# 입력 파일
input_file = 'filtered_tsunami_data.csv'
# 새 출력 파일
output_file = 'final_tsunami_data.csv'

try:
    # 1. CSV 파일 읽기
    df = pd.read_csv(input_file)

    # 2. 'Mo' 컬럼을 'Month'로 이름 변경
    df.rename(columns={'Mo': 'Month'}, inplace=True)

    # 3. 새 CSV 파일로 저장
    df.to_csv(output_file, index=False, encoding='utf-8-sig')

    print(f"컬럼명 변경 완료! '{output_file}' 로 저장되었습니다.")
    print("\n변경된 데이터 확인 (상위 5개):")
    print(df.head())

except FileNotFoundError:
    print(f"오류: '{input_file}'을(를) 찾을 수 없습니다.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

컬럼명 변경 완료! 'final_tsunami_data.csv' 로 저장되었습니다.

변경된 데이터 확인 (상위 5개):
   Year  Month  tsunami  mag  latitude  longitude  Tsunami Event Validity
0  2001      1        1  7.7    13.049    -88.660                       3
1  2001      6        1  8.4   -16.265    -73.641                       4
2  2001     10        1  6.1    52.630   -132.200                       4
3  2001     12        1  6.8    23.954    122.734                       4
4  2002      1        1  7.2   -17.600    167.856                       4


In [7]:
import pandas as pd

# 입력 파일
input_file = 'final_tsunami_data.csv'
# 새 출력 파일
output_file = 'real_final_tsunami_data.csv'
# 삭제할 컬럼 이름
column_to_drop = 'Tsunami Event Validity'

try:
    # 1. CSV 파일 읽기
    df = pd.read_csv(input_file)

    # 2. 'time' 컬럼이 있는지 확인
    if column_to_drop in df.columns:
        # 3. 'time' 컬럼 삭제
        # df.drop()을 사용하고 axis=1 (컬럼 기준)을 지정합니다.
        df_dropped = df.drop(columns=[column_to_drop])

        # 4. 새 CSV 파일로 저장
        df_dropped.to_csv(output_file, index=False, encoding='utf-8-sig')

        print(f"'{column_to_drop}' 컬럼을 성공적으로 삭제했습니다.")
        print(f"새 파일: '{output_file}'")
        print("\n삭제 후 컬럼 목록:")
        print(df_dropped.columns.to_list())

    else:
        print(f"오류: '{column_to_drop}' 컬럼이 파일에 없습니다.")

except FileNotFoundError:
    print(f"오류: '{input_file}' 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

'Tsunami Event Validity' 컬럼을 성공적으로 삭제했습니다.
새 파일: 'real_final_tsunami_data.csv'

삭제 후 컬럼 목록:
['Year', 'Month', 'tsunami', 'mag', 'latitude', 'longitude']


In [10]:
import pandas as pd

# --- 파일 설정 ---
# 비교할 두 파일
file1_earthquake = 'combined_earthquake_data.csv'
file2_tsunami = 'real_final_tsunami_data.csv'

# 결과(공통 행)를 저장할 파일
output_file = 'common_events_matched.csv'

# 비교할 기준이 되는 컬럼들
merge_columns = ['mag', 'latitude', 'longitude']

# 부동소수점(소수점) 비교를 위한 반올림 자릿수
# (예: 5.12와 5.120을 같게 처리하기 위함)
precision = 3
# --- --- ---

try:
    # 1. 두 CSV 파일 읽기
    df_eq = pd.read_csv(file1_earthquake)
    df_tsu = pd.read_csv(file2_tsunami)

    print(f"'{file1_earthquake}' 파일 로드 (총 {len(df_eq)} 행)")
    print(f"'{file2_tsunami}' 파일 로드 (총 {len(df_tsu)} 행)")

    # 2. 두 파일에 필요한 컬럼이 모두 있는지 확인
    missing_eq = [col for col in merge_columns if col not in df_eq.columns]
    missing_tsu = [col for col in merge_columns if col not in df_tsu.columns]

    error_found = False
    if missing_eq:
        print(f"오류: '{file1_earthquake}' 파일에 다음 컬럼이 없습니다: {missing_eq}")
        error_found = True
    if missing_tsu:
        print(f"오류: '{file2_tsunami}' 파일에 다음 컬럼이 없습니다: {missing_tsu}")
        error_found = True

    if not error_found:
        # 3. 비교 전 데이터 처리 (부동소수점 오차 방지)
        # 'mag', 'latitude', 'longitude' 컬럼을 숫자형으로 변환하고
        # 지정된 자릿수(3)에서 반올림하여 정확한 비교가 가능하게 함
        print(f"\n비교를 위해 기준 컬럼 {merge_columns} 값을 소수점 {precision}째 자리까지 표준화합니다...")

        # 원본을 유지하기 위해 .copy() 사용
        df_eq_processed = df_eq.copy()
        df_tsu_processed = df_tsu.copy()

        for col in merge_columns:
            # errors='coerce'는 숫자로 변환할 수 없는 값을 NaN(결측치)로 만듭니다.
            df_eq_processed[col] = pd.to_numeric(df_eq_processed[col], errors='coerce').round(precision)
            df_tsu_processed[col] = pd.to_numeric(df_tsu_processed[col], errors='coerce').round(precision)

        # 비교 컬럼 중 하나라도 NaN인 행은 삭제
        df_eq_processed.dropna(subset=merge_columns, inplace=True)
        df_tsu_processed.dropna(subset=merge_columns, inplace=True)

        # 4. 'inner' 조인(merge) 수행
        # 'how='inner'' 옵션은 두 데이터프레임에서
        # 'on=merge_columns' (mag, latitude, longitude) 값이 모두 일치하는 행만 남깁니다.
        # (이것이 SQL의 INNER JOIN과 동일한 기능입니다)
        common_events_df = pd.merge(
            df_eq_processed,
            df_tsu_processed,
            on=merge_columns,
            how='inner',
            suffixes=('_eq', '_tsu') # 두 파일에 이름이 겹치는 다른 컬럼(예: depth)을 구분하기 위함
        )

        # 5. 결과 저장 및 보고
        if len(common_events_df) > 0:
            common_events_df.to_csv(output_file, index=False, encoding='utf-8-sig')
            print(f"\n처리 완료! {len(common_events_df)} 개의 공통 행을 찾았습니다.")
            print(f"'{output_file}' 파일로 저장되었습니다.")
        else:
            print(f"\n두 파일 간에 {merge_columns} 값이 모두 일치하는 공통 행을 찾지 못했습니다.")

except FileNotFoundError as e:
    print(f"오류: 파일을 찾을 수 없습니다. {e}")
except Exception as e:
    print(f"알 수 없는 오류가 발생했습니다: {e}")

'combined_earthquake_data.csv' 파일 로드 (총 22507 행)
'real_final_tsunami_data.csv' 파일 로드 (총 130 행)

비교를 위해 기준 컬럼 ['mag', 'latitude', 'longitude'] 값을 소수점 3째 자리까지 표준화합니다...

처리 완료! 91 개의 공통 행을 찾았습니다.
'common_events_matched.csv' 파일로 저장되었습니다.


In [9]:
import pandas as pd
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import time

# --- 1단계: 로컬 파일 읽기 ---
file_path = 'query (7).csv'
print(f"Loading local file: {file_path}")

try:
    earthquake_list_df = pd.read_csv(file_path)
    print(f"Successfully loaded {len(earthquake_list_df)} events from file.")


    # --- 2단계: P/S파 데이터 수집 (수정됨) ---

    client = Client("IRIS")
    phase_data_list = []

    # (!! 테스트를 위해 처음 10개만 실행합니다. 전체는 .head(10)을 지우세요 !!)
    print(f"\nPart 2: Fetching P/S phase data for the first 10 events (Wide Search)...")

    for index, row in earthquake_list_df.head(10).iterrows():
        event_time = UTCDateTime(row['time'])

        print(f"\nProcessing Event at {event_time.date} {event_time.time} (Mag {row['mag']})")

        try:
            # === 여기가 수정된 부분입니다 ===
            # 시간, 위치, 규모의 검색 범위를 넓힙니다.

            catalog = client.get_events(
                starttime = event_time - 30,  # 지진 발생 30초 전
                endtime = event_time + 30,    # 지진 발생 30초 후
                latitude = row['latitude'],
                longitude = row['longitude'],
                maxradius = 1.0,              # ★중요: 반경 1도 (약 111km) 이내에서 검색
                minmagnitude = row['mag'] - 0.3, # 규모 오차범위 넓힘
                maxmagnitude = row['mag'] + 0.3, # 규모 오차범위 넓힘
                includearrivals = True       # P/S파 도착 정보 포함
            )

            if not catalog:
                print(f"  > No event found on IRIS matching this wide search.")
                continue

            # 검색된 이벤트 (가장 첫 번째 것)
            event = catalog[0]

            # 이벤트의 'preferred' (가장 공신력 있는) 진원 정보를 가져옵니다.
            origin = event.preferred_origin() or event.origins[0]

            p_arrivals_count = 0
            s_arrivals_count = 0

            # 진원 정보에 연결된 '도착(arrivals)' 목록을 확인합니다.
            if origin.arrivals:
                for arrival in origin.arrivals:
                    phase = arrival.phase
                    if phase.upper().startswith('P'):
                        p_arrivals_count += 1
                    elif phase.upper().startswith('S'):
                        s_arrivals_count += 1

            print(f"  > Found {p_arrivals_count} P-wave arrivals, {s_arrivals_count} S-wave arrivals.")

            phase_data_list.append({
                'id': row['id'], # id는 원래 USGS id를 그대로 저장
                'time': event_time,
                'latitude': row['latitude'],
                'longitude': row['longitude'],
                'mag': row['mag'],
                'p_wave_arrival_count': p_arrivals_count,
                's_wave_arrival_count': s_arrivals_count
            })

        except Exception as e:
            if "404" in str(e) or "No data available" in str(e):
                print(f"  > No phase data found for this event.")
            else:
                print(f"  > Could not retrieve phase data: {e}")

        time.sleep(1) # IRIS 서버는 USGS보다 더 엄격하므로 1초 대기

    # 7. 최종 결과를 DataFrame으로 변환
    phase_df = pd.DataFrame(phase_data_list)

    print("\n--- Final P/S Phase Data Summary (Test Run) ---")
    print(phase_df)

    # 8. 파일로 저장
    output_filename = "phase_data_from_query(7).csv"
    phase_df.to_csv(output_filename, index=False)
    print(f"\nSuccessfully saved data to {output_filename}")

# 1단계 파일 읽기 실패 시
except FileNotFoundError:
    print(f"Error: '{file_path}' not found. Make sure it's in the same directory as your script.")
except Exception as e:
    print(f"An error occurred: {e}")

Loading local file: query (7).csv
Successfully loaded 6768 events from file.

Part 2: Fetching P/S phase data for the first 10 events (Wide Search)...

Processing Event at 2012-12-31 20:02:04.160000 (Mag 5.2)
  > Found 0 P-wave arrivals, 0 S-wave arrivals.

Processing Event at 2012-12-31 01:15:18.770000 (Mag 5.1)
  > Found 0 P-wave arrivals, 0 S-wave arrivals.

Processing Event at 2012-12-30 20:49:27.830000 (Mag 5.2)
  > Found 0 P-wave arrivals, 0 S-wave arrivals.

Processing Event at 2012-12-30 12:21:21.870000 (Mag 5.3)
  > Found 0 P-wave arrivals, 0 S-wave arrivals.

Processing Event at 2012-12-30 11:33:43.030000 (Mag 5.2)
  > Found 0 P-wave arrivals, 0 S-wave arrivals.

Processing Event at 2012-12-30 07:56:52.070000 (Mag 5.0)
  > Found 0 P-wave arrivals, 0 S-wave arrivals.

Processing Event at 2012-12-30 06:44:41.480000 (Mag 5.2)
  > Found 0 P-wave arrivals, 0 S-wave arrivals.

Processing Event at 2012-12-29 23:05:24.200000 (Mag 5.2)
  > Found 0 P-wave arrivals, 0 S-wave arrivals.



In [10]:
import pandas as pd
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import time

# --- 1단계: 로컬 파일 읽기 ---
file_path = 'query (7).csv'
print(f"Loading local file: {file_path}")

try:
    earthquake_list_df = pd.read_csv(file_path)
    print(f"Successfully loaded {len(earthquake_list_df)} events from file.")


    # --- 2단계: P/S파 데이터 수집 (수정됨) ---

    # 2-1. IRIS 클라이언트로 접속
    client = Client("IRIS")

    phase_data_list = []

    # (!! 테스트를 위해 처음 10개만 실행합니다. 전체는 .head(10)을 지우세요 !!)
    print(f"\nPart 2: Fetching P/S phase data for the first 10 events (Wide Search)...")

    for index, row in earthquake_list_df.head(10).iterrows():
        event_time = UTCDateTime(row['time'])

        print(f"\nProcessing Event at {event_time.date} {event_time.time} (Mag {row['mag']})")

        try:
            # 2-2. 시간/위치/규모로 IRIS 서버에서 지진 검색
            catalog = client.get_events(
                starttime = event_time - 30,  # 지진 발생 30초 전
                endtime = event_time + 30,    # 지진 발생 30초 후
                latitude = row['latitude'],
                longitude = row['longitude'],
                maxradius = 1.0,              # 반경 1도 (약 111km) 이내
                minmagnitude = row['mag'] - 0.3,
                maxmagnitude = row['mag'] + 0.3,
                includearrivals = True       # ★ P/S파 도착 정보 포함
            )

            if not catalog:
                print(f"  > No event found on IRIS matching this wide search.")
                continue

            # 검색된 이벤트 (가장 첫 번째 것)
            event = catalog[0]

            p_arrivals_count = 0
            s_arrivals_count = 0

            # === 여기가 수정된 부분입니다 ===
            # 2-3. 'preferred_origin' 1개만 보는 대신, '모든' origins를 확인합니다.
            if event.origins:
                for origin in event.origins: # 모든 진원 정보를 반복
                    if origin.arrivals: # P/S파 도착 정보가 있다면
                        for arrival in origin.arrivals:
                            phase = arrival.phase
                            if phase.upper().startswith('P'):
                                p_arrivals_count += 1
                            elif phase.upper().startswith('S'):
                                s_arrivals_count += 1

            print(f"  > Found {p_arrivals_count} P-wave arrivals, {s_arrivals_count} S-wave arrivals.")

            phase_data_list.append({
                'id': row['id'], # id는 원래 USGS id를 그대로 저장
                'time': event_time,
                'latitude': row['latitude'],
                'longitude': row['longitude'],
                'mag': row['mag'],
                'p_wave_arrival_count': p_arrivals_count,
                's_wave_arrival_count': s_arrivals_count
            })

        except Exception as e:
            if "204" in str(e) or "No data available" in str(e):
                print(f"  > No phase data found for this event (204 No Content).")
                phase_data_list.append({
                    'id': row['id'], 'time': event_time, 'latitude': row['latitude'],
                    'longitude': row['longitude'], 'mag': row['mag'],
                    'p_wave_arrival_count': 0, 's_wave_arrival_count': 0
                })
            else:
                print(f"  > Could not retrieve phase data: {e}")

        time.sleep(1) # IRIS 서버에 부담을 주지 않기 위해 1초 대기

    # 7. 최종 결과를 DataFrame으로 변환
    phase_df = pd.DataFrame(phase_data_list)

    print("\n--- Final P/S Phase Data Summary (Test Run) ---")
    print(phase_df)

    # 8. 파일로 저장
    output_filename = "phase_data_from_query(7).csv"
    phase_df.to_csv(output_filename, index=False)
    print(f"\nSuccessfully saved data to {output_filename}")

# 1단계 파일 읽기 실패 시
except FileNotFoundError:
    print(f"Error: '{file_path}' not found. Make sure it's in the same directory as your script.")
except Exception as e:
    print(f"An error occurred: {e}")

Loading local file: query (7).csv
Successfully loaded 6768 events from file.

Part 2: Fetching P/S phase data for the first 10 events (Wide Search)...

Processing Event at 2012-12-31 20:02:04.160000 (Mag 5.2)
  > Found 0 P-wave arrivals, 0 S-wave arrivals.

Processing Event at 2012-12-31 01:15:18.770000 (Mag 5.1)
  > Found 0 P-wave arrivals, 0 S-wave arrivals.

Processing Event at 2012-12-30 20:49:27.830000 (Mag 5.2)
  > Found 0 P-wave arrivals, 0 S-wave arrivals.

Processing Event at 2012-12-30 12:21:21.870000 (Mag 5.3)
  > Found 0 P-wave arrivals, 0 S-wave arrivals.

Processing Event at 2012-12-30 11:33:43.030000 (Mag 5.2)
  > Found 0 P-wave arrivals, 0 S-wave arrivals.

Processing Event at 2012-12-30 07:56:52.070000 (Mag 5.0)
  > Found 0 P-wave arrivals, 0 S-wave arrivals.

Processing Event at 2012-12-30 06:44:41.480000 (Mag 5.2)
  > Found 0 P-wave arrivals, 0 S-wave arrivals.

Processing Event at 2012-12-29 23:05:24.200000 (Mag 5.2)
  > Found 0 P-wave arrivals, 0 S-wave arrivals.



In [ ]:
import requests
import pandas as pd
import io

base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
all_earthquakes = []

print("Part 1: Downloading Earthquake List (2001-2022)...")

# 2001년부터 2022년까지 1년 단위로 반복
for year in range(2001, 2023):
    print(f"Fetching data for {year}...")
    params = {
        'format': 'csv',
        'starttime': f'{year}-01-01',
        'endtime': f'{year}-12-31',
        'minmagnitude': 5.0,  # P/S파 분석을 위해 최소 규모 5.0 이상으로 제한
        'orderby': 'time',
        'eventtype': 'earthquake'
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            csv_data = io.StringIO(response.text)
            df_year = pd.read_csv(csv_data)
            all_earthquakes.append(df_year)
            print(f"  > Success: Found {len(df_year)} events in {year}.")
        else:
            print(f"  > Failed for {year}. Status: {response.status_code}")

    except Exception as e:
        print(f"  > Error during request for {year}: {e}")

# 모든 연도별 데이터를 하나로 합치기
earthquake_list_df = pd.concat(all_earthquakes, ignore_index=True)
print(f"\nTotal events found from 2001-2022 (Mag 5.0+): {len(earthquake_list_df)}")
print(earthquake_list_df.head())

In [ ]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import time

# 1. IRIS 클라이언트 접속
client = Client("IRIS")

# 2. P파, S파 도착 시간을 저장할 리스트
phase_data_list = []

# 3. 1단계에서 만든 목록(earthquake_list_df)을 사용
# (!! 테스트를 위해 처음 10개만 실행해 보세요 !!)
# (전부 실행하려면 [:] 로 바꾸세요)
for index, row in earthquake_list_df.head(10).iterrows():
    event_id = row['id']
    event_time = UTCDateTime(row['time'])

    print(f"\nProcessing Event ID: {event_id} ({event_time.date})")

    try:
        # 4. 'eventid'를 사용해 'phase data'가 포함된 이벤트 정보 요청
        catalog = client.get_events(eventid=event_id, includeallorigins=True, includeallphases=True)

        # 5. 첫 번째 이벤트의 정보 (보통 하나만 나옴)
        event = catalog[0]

        # 6. 모든 도착 정보(arrivals)에서 P파, S파 정보 추출
        p_arrivals = 0
        s_arrivals = 0

        if event.picks:
            for pick in event.picks:
                phase_hint = pick.phase_hint
                if phase_hint in ['P', 'p', 'Pn', 'Pg']:
                    p_arrivals += 1
                elif phase_hint in ['S', 's', 'Sn', 'Sg']:
                    s_arrivals += 1

        print(f"  > Found {p_arrivals} P-wave arrivals, {s_arrivals} S-wave arrivals.")

        phase_data_list.append({
            'id': event_id,
            'time': event_time,
            'latitude': row['latitude'],
            'longitude': row['longitude'],
            'mag': row['mag'],
            'p_wave_arrival_count': p_arrivals, # P파가 관측된 관측소 수
            's_wave_arrival_count': s_arrivals  # S파가 관측된 관측소 수
        })

    except Exception as e:
        print(f"  > Could not retrieve phase data for {event_id}: {e}")

    # (API 서버에 부담을 주지 않기 위해 잠시 대기)
    time.sleep(0.5)

# 7. 최종 결과를 DataFrame으로 변환
phase_df = pd.DataFrame(phase_data_list)

print("\n--- Final P/S Phase Data Summary ---")
print(phase_df)

# 파일로 저장
phase_df.to_csv("earthquake_phase_data_2001-2022.csv", index=False)

In [3]:
import pandas as pd
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import time

# 1. IRIS 클라이언트로 접속
client = Client("IRIS")

def get_detailed_phases(event_id, event_mag):
    """지진 ID를 받아서 상세 P/S파 개수를 세는 함수"""

    print(f"\nProcessing Event ID: {event_id} (Mag {event_mag})")
    p_arrivals_count = 0
    s_arrivals_count = 0

    try:
        # 'eventid' 파라미터를 사용하여 상세 정보 요청
        detailed_catalog = client.get_events(
            eventid=event_id,
            includearrivals=True
        )

        if not detailed_catalog:
            print("  > Could not fetch detailed catalog for this event.")
            return p_arrivals_count, s_arrivals_count

        # "상세 정보"에서 'picks'가 아니라 'origins' 안의 'arrivals'를 확인
        detailed_event = detailed_catalog[0]

        # === THIS IS THE NEW LOGIC ===
        if detailed_event.origins:
            # 모든 origin을 다 확인
            for origin in detailed_event.origins:
                if origin.arrivals:
                    for arrival in origin.arrivals:
                        phase = arrival.phase
                        if phase and phase.upper().startswith('P'):
                            p_arrivals_count += 1
                        elif phase and phase.upper().startswith('S'):
                            s_arrivals_count += 1
        # =============================

        print(f"  > SUCCESS: Found {p_arrivals_count} P-wave arrivals, {s_arrivals_count} S-wave arrivals.")

    except Exception as e:
        if "204" in str(e) or "No data available" in str(e):
            print(f"  > No detailed phase data found for this event (204 No Content).")
        else:
            print(f"  > Error fetching details: {e}")

    return p_arrivals_count, s_arrivals_count

# --- 테스트 1: 님이 주신 2022년 데이터 (결과가 0일 것으로 예상) ---
print("--- TEST 1: Your 2022 Event ---")
test_1_id = "11640145" # 2022년 목록 첫 번째 ID
test_1_mag = 5.1
p_count_1, s_count_1 = get_detailed_phases(test_1_id, test_1_mag)

time.sleep(1) # 1초 대기

# --- 테스트 2: 2011년 동일본 대지진 (결과가 0이 아니어야 함) ---
print("\n--- TEST 2: 2011 Japan Earthquake (Tohoku) ---")
# 2011년 동일본 대지진의 IRIS ID (GCMT 카탈로그 기준)
test_2_id = "1418931"
test_2_mag = 9.1
p_count_2, s_count_2 = get_detailed_phases(test_2_id, test_2_mag)


# --- 최종 결과 요약 ---
print("\n--- TEST RESULTS ---")
print(f"Test 1 (Your 2022 Event, ID: {test_1_id}): P={p_count_1}, S={s_count_1}")
print(f"Test 2 (2011 Japan Event, ID: {test_2_id}): P={p_count_2}, S={s_count_2}")

if p_count_1 == 0 and p_count_2 > 0:
    print("\n[결론]")
    print("코드는 정상적으로 작동합니다.")
    print("하지만 2022년의 규모 5.0 지진들은 IRIS 데이터베이스에 P/S파 'picks' 정보가 등록되어 있지 않습니다 (0개가 맞습니다).")
else:
    print("\n[결론]")
    print("테스트 2(2011년 지진)에서도 0이 나온다면, 코드나 네트워크에 다른 문제가 있는 것입니다.")

--- TEST 1: Your 2022 Event ---

Processing Event ID: 11640145 (Mag 5.1)
  > SUCCESS: Found 0 P-wave arrivals, 0 S-wave arrivals.

--- TEST 2: 2011 Japan Earthquake (Tohoku) ---

Processing Event ID: 1418931 (Mag 9.1)
  > SUCCESS: Found 0 P-wave arrivals, 0 S-wave arrivals.

--- TEST RESULTS ---
Test 1 (Your 2022 Event, ID: 11640145): P=0, S=0
Test 2 (2011 Japan Event, ID: 1418931): P=0, S=0

[결론]
테스트 2(2011년 지진)에서도 0이 나온다면, 코드나 네트워크에 다른 문제가 있는 것입니다.


In [2]:
import obspy
from obspy.clients.fdsn import Client
import inspect # 경로 추적을 위한 모듈
import time

print(f"--- 현재 obspy 버전: {obspy.__version__} ---")

# ---!!! 결정적인 진단 코드 !!!---
# Client 클래스가 로드된 실제 파일 경로를 출력합니다.
# 이 경로가 1.4.2가 설치된 경로(예: .../site-packages/obspy/clients/fdsn/client.py)여야 합니다.
try:
    print(f"--- Client 클래스 파일 위치: {inspect.getfile(Client)} ---")
except Exception as e:
    print(f"--- Client 클래스 파일 위치 추적 실패: {e} ---")

# ---!!! get_picks 함수가 존재하는지 다시 확인 ---
if hasattr(Client("IRIS"), 'get_picks'):
    print("--- 진단: get_picks() 함수가 정상적으로 존재합니다. ---")
else:
    print("--- !!!심각한 오류: get_picks() 함수가 여전히 없습니다. 설치가 완전히 꼬였습니다.!!! ---")

# -----------------------------------------------

# 이전에 사용했던 'get_picks' 함수 (이제 작동해야 함)
def get_picks_for_event(event_id, event_mag):
    print(f"\nProcessing Event ID: {event_id} (Mag {event_mag}) using get_picks()")
    p_pick_count = 0
    s_pick_count = 0

    client = Client("IRIS") # 객체 새로 생성

    try:
        picks_catalog = client.get_picks(
            eventid=event_id
        )

        if not picks_catalog:
            print(f"  > No associated 'Pick' data found for Event ID {event_id}.")
            return p_pick_count, s_pick_count

        print(f"  > SUCCESS: Found {len(picks_catalog)} total 'Picks' associated with this event.")

        for pick in picks_catalog:
            phase = pick.phase_hint
            if phase:
                phase_upper = phase.upper()
                if phase_upper.startswith('P'):
                    p_pick_count += 1
                elif phase_upper.startswith('S'):
                    s_pick_count += 1

        print(f"  > P-Picks: {p_pick_count}")
        print(f"  > S-Picks: {s_pick_count}")

    except AttributeError as ae:
        print(f"  > !!! AttributeError 발생: {ae}")
        print("  > (원인: 환경이 여전히 꼬여있습니다. 위 '강제 재설치'를 다시 시도하세요.)")
    except Exception as e:
        if "204" in str(e) or "No data available" in str(e):
            print(f"  > No associated 'Pick' data found for Event ID {event_id} (204 No Content).")
        else:
            print(f"  > Error fetching picks: {e}")

    return p_pick_count, s_pick_count

# --- 테스트 1: 2022년 데이터 ---
print("\n--- TEST 1: Your 2022 Event (using get_picks) ---")
test_1_id = "11640145"
test_1_mag = 5.1
p_count_1, s_count_1 = get_picks_for_event(test_1_id, test_1_mag)

time.sleep(1)

# --- 테스트 2: 2011년 동일본 대지진 ---
print("\n--- TEST 2: 2011 Japan Earthquake (using get_picks) ---")
test_2_id = "1418931"
test_2_mag = 9.1
p_count_2, s_count_2 = get_picks_for_event(test_2_id, test_2_mag)

# --- 최종 결과 요약 ---
print("\n--- TEST RESULTS (using get_picks) ---")
print(f"Test 1 (Your 2022 Event, ID: {test_1_id}): P={p_count_1}, S={s_count_1}")
print(f"Test 2 (2011 Japan Event, ID: {test_2_id}): P={p_count_2}, S={s_count_2}")

--- 현재 obspy 버전: 1.4.2 ---
--- Client 클래스 파일 위치: C:\Users\admin\Desktop\3-team-project\team_project_python\.venv\Lib\site-packages\obspy\clients\fdsn\client.py ---
--- !!!심각한 오류: get_picks() 함수가 여전히 없습니다. 설치가 완전히 꼬였습니다.!!! ---

--- TEST 1: Your 2022 Event (using get_picks) ---

Processing Event ID: 11640145 (Mag 5.1) using get_picks()
  > !!! AttributeError 발생: 'Client' object has no attribute 'get_picks'
  > (원인: 환경이 여전히 꼬여있습니다. 위 '강제 재설치'를 다시 시도하세요.)

--- TEST 2: 2011 Japan Earthquake (using get_picks) ---

Processing Event ID: 1418931 (Mag 9.1) using get_picks()
  > !!! AttributeError 발생: 'Client' object has no attribute 'get_picks'
  > (원인: 환경이 여전히 꼬여있습니다. 위 '강제 재설치'를 다시 시도하세요.)

--- TEST RESULTS (using get_picks) ---
Test 1 (Your 2022 Event, ID: 11640145): P=0, S=0
Test 2 (2011 Japan Event, ID: 1418931): P=0, S=0
